# Cloud  Backed Earth Engine Tables

In [1]:
import pandas as pd
from google.cloud import storage
import os
import json

## Start an authorized session

To be able to make an Earth Engine asset in your user folder, you need to be able to authenticate as yourself when you make the request.  You can use credentials from the Earth Engine authenticator to start an [`AuthorizedSession`](https://google-auth.readthedocs.io/en/master/reference/google.auth.transport.requests.html#google.auth.transport.requests.AuthorizedSession).  You can then use the `AuthorizedSession` to send requests to Earth Engine.

In [ ]:
import ee
from google.auth.transport.requests import AuthorizedSession

#ee.Authenticate()  #  or 
!earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

## Get table information from gcp
Get list of tables and their respective uris from the gcp data bucket


In [ ]:
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):
    

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    # Note: The call returns a response only when the iterator is consumed.
    blob_list = []
    for blob in blobs:
        blob_list.append(blob.name)

    if delimiter:
        print("Prefixes:")
        for prefix in blobs.prefixes:
            blob_list.append([prefix])
    
    return blob_list


In [ ]:
table_name = 'nhdplus' 

In [ ]:
BUCKET_NAME = 'swhm-bq-exports'
blobsout = list_blobs_with_prefix(BUCKET_NAME,'cocs/'+table_name+'/'+table_name)

In [ ]:
df = pd.DataFrame(blobsout, columns=['blob_name'])
df['uri'] = 'gs://'+BUCKET_NAME+'/'+df['blob_name']
(df)

In [ ]:

file_names = [os.path.splitext(os.path.basename(file_path))[0] for file_path in df['blob_name']]
df["asset_name"] = table_name

df

In [ ]:
asset_name = df['asset_name'][0]
uri = df['uri'][0]
cmd = 'earthengine upload table --asset_id=projects/ee-swhm/assets/production_feature_collections/{} {}'.format(asset_name,uri) 
!{cmd}